# Solving Squad Selection Problem with *sasoptpy*

In many team sports, such as soccer, basketball, and e-sports, choosing a squad among potential players is a common task. In the following example, consider a generic problem, where the decision maker is trying to sign 3 players among hundreds of applicants. The objective is to maximize the total rating of the squad.

The problem summary is as follows:

- Data:
  - List of players, their attributes, desired position(s), and contract price
  - List of positions and the weight of each attribute
  - A budget limit
- Decision:
  - Choosing a player to sign for each position
- Constraint:
  - Total signing cost should not exceed the budget
  - Players can only play in their desired position

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../..'))

In [2]:
import pandas as pd
import numpy as np
import sasoptpy as so
print('sasoptpy version:', so.__version__)
from sasoptpy.actions import read_data
import swat
import math
from functools import lru_cache

sasoptpy version: 1.0.0-beta.3


In [3]:
# Options for numpy and pandas
np.random.seed(42)
pd.options.display.max_columns = 10

In [4]:
# Random problem generation parameters
num_candidates = 50
num_positions = 3
BUDGET = 250000

In [5]:
# Generating random problem data
candidate_rating = pd.DataFrame(
    np.random.randint(30, 100,size=(num_candidates, num_positions)),
    columns=[f'Position {i+1}' for i in range(num_positions)]).set_index(
    pd.Index([f'Candidate {i+1:02}' for i in range(num_candidates)]))

# Players wants to play their best position, or if the rating is greater than 75
def filter_for_request(stats):
    return stats >= min(75, stats.max())

request = candidate_rating.apply(filter_for_request, axis=1).astype(int)

In [6]:
# Player ratings
candidate_rating

,Position 1,Position 2,Position 3
Candidate 01,81,44,90
Candidate 02,50,53,32
Candidate 03,51,82,31
Candidate 04,59,67,31
Candidate 05,93,89,50
Candidate 06,62,87,51
Candidate 07,78,88,71
Candidate 08,89,44,91
Candidate 09,91,76,91
Candidate 10,80,84,93


In [7]:
# Positions where players can play
request

,Position 1,Position 2,Position 3
Candidate 01,1,0,1
Candidate 02,0,1,0
Candidate 03,0,1,0
Candidate 04,0,1,0
Candidate 05,1,1,0
Candidate 06,0,1,0
Candidate 07,1,1,0
Candidate 08,1,0,1
Candidate 09,1,1,1
Candidate 10,1,1,1


**Note**

You need to have access to a CAS server (SAS Viya installation) to be able to run this example. You can pass host address and the port as an environment variable as follows:

In [8]:
# Create SAS Viya connection
s = swat.CAS(os.getenv('ORCASHOST'), os.getenv('ORCASPORT'))

In [9]:
# Uplaod data
data = pd.concat([candidate_rating.stack(), request.stack()], axis=1)
data.reset_index(inplace=True)
data.columns = ['Candidate', 'Position', 'Rating', 'CanPlay']
rating_data = s.upload_frame(data, casout={'name': 'candidate_position_data', 'replace': True})

price_raw = candidate_rating.apply(lambda x: np.random.randint(x.max(), 100)*1000, axis=1)
price = price_raw.reset_index()
price.columns = ['Candidate', 'Price']
price_data = s.upload_frame(price, casout={'name': 'candidate_price', 'replace': True})

NOTE: Cloud Analytic Services made the uploaded file available as table CANDIDATE_POSITION_DATA in caslib CASUSERHDFS(sercay).
NOTE: The table CANDIDATE_POSITION_DATA has been created in caslib CASUSERHDFS(sercay) from binary data uploaded to Cloud Analytic Services.
NOTE: Cloud Analytic Services made the uploaded file available as table CANDIDATE_PRICE in caslib CASUSERHDFS(sercay).
NOTE: The table CANDIDATE_PRICE has been created in caslib CASUSERHDFS(sercay) from binary data uploaded to Cloud Analytic Services.


In [10]:
# Reset counters
so.reset()

# Model
m = so.Model(name='squad_selection', session=s)

# Index sets and parameters
C = m.add_set(name='CANDIDATES', settype=so.string)
P = m.add_set(name='POSITIONS', settype=so.string)
CP = m.add_set(name='CANDIDATE_POSITIONS', settype=[so.string, so.string])
rating = m.add_parameter(CP, name='Rating')
can_play = m.add_parameter(CP, name='CanPlay')
price = m.add_parameter(C, name='Price')
budget_limit = m.add_parameter(name='budget_limit', value=BUDGET)

# Read server data into problem
m.include(
    read_data(table=rating_data, index={'target': CP, 'key': ['candidate', 'position']},
             columns=[rating, can_play])
)
m.include(
    read_data(table=price_data, index={'target': C, 'key': ['Candidate']},
             columns=[price])
)
m.include(read_data(table=rating_data, index={'target': P, 'key': 'position'}, columns=[]))

NOTE: Initialized model squad_selection.


In [11]:
# Create variables
hire_candidate_for_position = m.add_variables(CP, name='hire', vartype=so.binary)
player_pair = m.add_variables(CP, CP, name='player_pos_pair', vartype=so.binary)

In [12]:
# Total rating is quadratic function of selected player ratings, linearized using linking constraints
total_rating = \
    so.Objective(
        so.expr_sum(
            so.expr_sum(
                player_pair[cp1, cp2] * rating[cp1] * rating[cp2]
                for cp2 in CP if cp2.sym.under_condition((cp2[0] != cp1[0]) & (cp2[1] != cp1[1]))) for cp1 in CP),
    name='total_rating', sense=so.maximize)
m.include(total_rating)

In [13]:
# Constraints

# Linking constraints
m.add_constraints(
    (player_pair[cp1, cp2] >= hire_candidate_for_position[cp1] + hire_candidate_for_position[cp2] - 1 for cp1 in CP for cp2 in CP),
    name='pair_lb')
m.add_constraints(
    (player_pair[cp1, cp2] <= hire_candidate_for_position[cp1] for cp1 in CP for cp2 in CP), name='pair_ub1')
m.add_constraints(
    (player_pair[cp1, cp2] <= hire_candidate_for_position[cp2] for cp1 in CP for cp2 in CP), name='pair_ub2')
# One player can only play at preferred positions
m.add_constraints((hire_candidate_for_position[cp] <= can_play[cp] for cp in CP), name='position_limit');
# Total signing cost should be less than budget
m.add_constraint(so.expr_sum(hire_candidate_for_position[cp] * price[cp[0]] for cp in CP) <= budget_limit, name='budget_con')
# A position should be signed with exactly one player
m.add_constraints((so.expr_sum(hire_candidate_for_position[c,p] for c in C) == 1 for p in P), name='position_con')
# A player can be signed for at most one position
m.add_constraints((so.expr_sum(hire_candidate_for_position[c,p] for p in P) <= 1 for c in C), name='player_con');

In [14]:
# Solve using MILP solver
m.solve(options={'with': 'milp'}, verbose=True)

NOTE: Added action set 'optimization'.
NOTE: Converting model squad_selection to OPTMODEL.
   set <str> CANDIDATES;
   set <str> POSITIONS;
   set <str, str> CANDIDATE_POSITIONS;
   num Rating {{CANDIDATE_POSITIONS}};
   num CanPlay {{CANDIDATE_POSITIONS}};
   num Price {{CANDIDATES}};
   num budget_limit = 250000;
   read data CANDIDATE_POSITION_DATA into CANDIDATE_POSITIONS=[candidate position] Rating CanPlay;
   read data CANDIDATE_PRICE into CANDIDATES=[Candidate] Price;
   read data CANDIDATE_POSITION_DATA into POSITIONS=[position] ;
   var hire {{CANDIDATE_POSITIONS}} binary;
   var player_pos_pair {{CANDIDATE_POSITIONS}, {CANDIDATE_POSITIONS}} binary;
   max total_rating = sum {<o19, o21> in CANDIDATE_POSITIONS} (sum {<o25, o27> in CANDIDATE_POSITIONS: ((o25 NE o19) and (o27 NE o21))} (player_pos_pair[o19, o21, o25, o27] * Rating[o19, o21] * Rating[o25, o27]));
   con pair_lb_o38_o44 {<o39, o41> in CANDIDATE_POSITIONS, <o45, o47> in CANDIDATE_POSITIONS} : player_pos_pair[o39, o4

,i,var,value,lb,ub,rc
0,1.0,"hire['Candidate 01','Position 1']",-0.0,-0.0,1.0,NaN
1,2.0,"hire['Candidate 01','Position 2']",-0.0,-0.0,1.0,NaN
2,3.0,"hire['Candidate 01','Position 3']",0.0,-0.0,1.0,NaN
3,4.0,"hire['Candidate 02','Position 1']",-0.0,-0.0,1.0,NaN
4,5.0,"hire['Candidate 02','Position 2']",-0.0,-0.0,1.0,NaN
...,...,...,...,...,...,...
22645,22646.0,"player_pos_pair['Candidate 50','Position 3','C...",-0.0,-0.0,1.0,NaN
22646,22647.0,"player_pos_pair['Candidate 50','Position 3','C...",-0.0,-0.0,1.0,NaN
22647,22648.0,"player_pos_pair['Candidate 50','Position 3','C...",-0.0,-0.0,1.0,NaN
22648,22649.0,"player_pos_pair['Candidate 50','Position 3','C...",-0.0,-0.0,1.0,NaN


In [15]:
# Get solution and filter for selected values
soln = so.get_solution_table(hire_candidate_for_position)
chosen = soln[soln['hire'] > 0.5]
chosen

,hire
"(Candidate 14, Position 2)",1.0
"(Candidate 16, Position 1)",1.0
"(Candidate 48, Position 3)",1.0


In [16]:
# Print candidate, position, rating, and contract value
for i in chosen.index.tolist():
    print(f'{i[0]} is hired for {i[1]}, player\'s rating at the position: {candidate_rating.loc[i]}, contract price: {price_raw.loc[i[0]]}')

print(f'Budget: {budget_limit.get_value()} Team\'s rating: {math.sqrt(m.get_objective_value()/2/num_positions):.2f}')

Candidate 14 is hired for Position 2, player's rating at the position: 82, contract price: 82000
Candidate 16 is hired for Position 1, player's rating at the position: 76, contract price: 77000
Candidate 48 is hired for Position 3, player's rating at the position: 88, contract price: 90000
Budget: 250000 Team's rating: 81.93


## Visualizing the solution

The solution to the optimization problem is visualized below. Here, you can benefit from ipywidgets Python package to display interactive controls.

You can also use the native caching mechanism of Python. Since optimization is an expensive task, you might find easier to cache previous solutions.

In [17]:
# Update the budget limit and resolve the problem
# LRU_CACHE ensures that if we have solved the problem with this budget, cached result will be returned

@lru_cache(maxsize=None)
def solve_squad_selection_problem(budget):
    budget_limit.set_value(budget)
    m.solve(options={'with': 'milp'})
    soln = so.get_solution_table(hire_candidate_for_position)
    chosen = soln[soln['hire'] > 0.5]
    return chosen

In [18]:
@lru_cache(maxsize=None)
def wrapper(budget):
    opt_result = solve_squad_selection_problem(budget)
    result_table = [0 for i in range(num_positions)]
    # Print candidate, position, rating, and contract value
    for i in opt_result.index.tolist():
        result_table[int(i[1].split(' ')[1])-1] = (i[0], candidate_rating.loc[i], price_raw.loc[i[0]])

    total_rating = math.sqrt(m.get_objective_value()/2/num_positions)
    return result_table, total_rating

In [19]:
from ipywidgets import Layout, Box, VBox, HBox, Label, IntSlider, HTML, Image
from ipycanvas import Canvas
from math import pi

In [20]:
# Prepare the ipywidget

# Left side

budget_slider = IntSlider(
    min=240000, max=300000, step=10000, description='Budget',
    disabled=False, continuous_update=False, orientation='vertical',
    layout=Layout(margin='auto 0', height='300px', width='100px', font_weight='3')
)
slider_box = VBox([budget_slider], layout=Layout(width='25%', align_items='center'))

# Divider
divider = VBox(layout=Layout(border='1px solid black', margin='10px', width='0px'))

load_img = HTML(value='<span style="height: 24px">&nbsp;</span>')
message_label = Label('')
message_box = VBox([load_img, message_label], layout=Layout(margin='0 auto', align_items='center'))

# Right side


def fill_canvas(ratings=None, details=None):
    canvas = Canvas(width=500, height=300, layout=Layout(margin='auto auto'))
    canvas.stroke_style = 'black'
    canvas.text_align = 'left'
    canvas.text_baseline = 'middle'
    canvas.font = '14px trebuchet ms'

    colors = ['#0070c0', '#70ad47', '#6f30a0']
    xpos = [250, 125, 375]
    ypos = [50, 250, 250]
    if ratings is None:
        ratings = ['P1', 'P2', 'P3']

    if details is not None:
        dx = [125, 0, 410]
        dy = [50, 250, 250]
        for j in range(3):
            canvas.fill_text(details[j][0], dx[j], dy[j]-6)
            canvas.fill_text(f'${int(details[j][2]/1000)}K', dx[j], dy[j]+6)
    
    canvas.text_align = 'center'
    canvas.font = '24px trebuchet ms'
    canvas.line_width = 3
    for i in range(3):
        canvas.begin_path()
        canvas.move_to(xpos[i], ypos[i])
        canvas.line_to(xpos[0 if i+1>=3 else i+1], ypos[0 if i+1>=3 else i+1])
        canvas.stroke()

    canvas.line_width = 2
    for i in range(3):
        canvas.fill_style = colors[i]
        canvas.fill_arc(xpos[i], ypos[i], radius=30, start_angle=0, end_angle=2*pi)
        canvas.stroke_arc(xpos[i], ypos[i], radius=29, start_angle=0, end_angle=2*pi)
        canvas.fill_style = 'white'
        canvas.fill_text(ratings[i], xpos[i], ypos[i])
    
    graph_box.children = [canvas,]


graph_box = HBox(layout=Layout(min_height='305px'))
fill_canvas()

budget_value = '---'
rating_value = '--.--'
label1 = HTML(value = "<font size='3pt'><b>Budget</b>:</font>")
label2 = HTML(value = "<font size='3pt'><b>Rating</b>:</font>")
label_box = HBox([VBox([label1, label2], layout=Layout(margin='auto auto', width='125px'))],
                 layout=Layout())

visual_box = VBox([message_box, graph_box, label_box], layout=Layout(width='75%'))


widget = Box([slider_box, divider, visual_box], layout=Layout(width='100%', min_height='80px', border='1px solid black'))

def update_outcome(value):
    # Stop animation
    budget_slider.disabled = True
    load_img.value="<img src='loader.svg' />"
    message_label.value = 'Optimizing.. Please wait..'
    # Call wrapper
    result = wrapper(value.new)
    # Update values
    ratings = [i[1] for i in result[0]]
    fill_canvas(ratings, result[0])
    label1.value = f"<font size='3pt'><b>Budget</b>: ${int(value.new/1000)}K</font>"
    rating_value = result[1]
    label2.value = f"<font size='3pt'><b>Rating</b>: {rating_value:.2f}</font>"
    # Resume animation
    budget_slider.disabled = False
    load_img.value = '<span style="height: 24px">&nbsp;</span>'
    message_label.value = ''

budget_slider.observe(update_outcome, 'value')

In [21]:
widget

Box(children=(VBox(children=(IntSlider(value=240000, continuous_update=False, description='Budget', layout=Lay…

NOTE: Added action set 'optimization'.
NOTE: Converting model squad_selection to OPTMODEL.
NOTE: Submitting OPTMODEL code to CAS server.
NOTE: There were 150 rows read from table 'CANDIDATE_POSITION_DATA' in caslib 'CASUSERHDFS(sercay)'.
NOTE: There were 50 rows read from table 'CANDIDATE_PRICE' in caslib 'CASUSERHDFS(sercay)'.
NOTE: 147 duplicate keys were read.
NOTE: There were 150 rows read from table 'CANDIDATE_POSITION_DATA' in caslib 'CASUSERHDFS(sercay)'.
NOTE: Problem generation will use 8 threads.
NOTE: The problem has 22650 variables (0 free, 0 fixed).
NOTE: The problem has 22650 binary and 0 integer variables.
NOTE: The problem has 67704 linear constraints (45200 LE, 3 EQ, 22501 GE, 0 range).
NOTE: The problem has 157950 linear constraint coefficients.
NOTE: The problem has 0 nonlinear constraints (0 LE, 0 EQ, 0 GE, 0 range).
NOTE: The initial MILP heuristics are applied.
NOTE: The MILP presolver value AUTOMATIC is applied.
NOTE: The MILP presolver removed 21656 variables an

NOTE: The output table 'SOLUTION' in caslib 'CASUSERHDFS(sercay)' has 22650 rows and 6 columns.
NOTE: The output table 'DUAL' in caslib 'CASUSERHDFS(sercay)' has 67704 rows and 4 columns.
NOTE: The CAS table 'solutionSummary' in caslib 'CASUSERHDFS(sercay)' has 18 rows and 4 columns.
NOTE: The CAS table 'problemSummary' in caslib 'CASUSERHDFS(sercay)' has 20 rows and 4 columns.
NOTE: Added action set 'optimization'.
NOTE: Converting model squad_selection to OPTMODEL.
NOTE: Submitting OPTMODEL code to CAS server.
NOTE: There were 150 rows read from table 'CANDIDATE_POSITION_DATA' in caslib 'CASUSERHDFS(sercay)'.
NOTE: There were 50 rows read from table 'CANDIDATE_PRICE' in caslib 'CASUSERHDFS(sercay)'.
NOTE: 147 duplicate keys were read.
NOTE: There were 150 rows read from table 'CANDIDATE_POSITION_DATA' in caslib 'CASUSERHDFS(sercay)'.
NOTE: Problem generation will use 8 threads.
NOTE: The problem has 22650 variables (0 free, 0 fixed).
NOTE: The problem has 22650 binary and 0 integer v

                0        1      3  41158.0000000  69866.2925170   41.09%       3
                0        1      3  41158.0000000  69777.6438356   41.02%       4
                0        1      3  41158.0000000  69527.6000000   40.80%       4
                0        1      3  41158.0000000  69491.3333333   40.77%       4
                0        1      3  41158.0000000  69454.2537563   40.74%       4
                0        1      3  41158.0000000  69378.5908257   40.68%       4
                0        1      3  41158.0000000  69332.7258065   40.64%       4
                0        1      3  41158.0000000  69323.9393939   40.63%       4
                0        1      3  41158.0000000  69316.1538462   40.62%       4
                0        1      3  41158.0000000  69295.5702420   40.61%       4
                0        1      3  41158.0000000  69275.7048518   40.59%       4
NOTE: The MILP solver added 42 cuts with 303 cut coefficients at the root.
                2        2      4 